In [1]:
import os
import itertools
import codecs
import re
import datetime
# import cairocffi as cairo
# import editdistance
import numpy as np
from scipy import ndimage
# import pylab
import matplotlib.pyplot as plt
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks
import pathlib
import cv2

Using TensorFlow backend.


In [2]:
from icp_factory import GenCaptcha

gencaptcha = GenCaptcha()

class TextImageGenerator(keras.callbacks.Callback):
    # 所有可能字符
    LABELS = '0123456789abcdefghijklmnopqrstuvwxyz '
    
    def __init__(self, train_path, validate_path, img_w, img_h, channel, downsample_factor, absolute_max_string_len=6):
        """
        Args:
            train_path: 训练数据路径
            validate_path: 验证图片路径
            img_w:
            img_h:
            downsample_factor: TODO 未知
            absolute_max_string_len: 最大字符串长度
        """
        self.img_w = img_w
        self.img_h = img_h
        self.channel = channel
        self.train_path = train_path
        self.validate_path = validate_path
        self.downsample_factor = downsample_factor
        self.blank_label = self.get_output_size() - 1
        self.absolute_max_string_len = absolute_max_string_len
        # 数据
        self.train_imgs = self.get_all_imgs(self.train_path)
        self.validate_imgs = self.get_all_imgs(self.validate_path)
        self.cur_idx = 0
        
        np.random.shuffle(self.train_imgs)
        np.random.shuffle(self.validate_imgs)
    
    def get_all_imgs(self, path):
#         p = pathlib.Path(path)
        # jpg or png
#         return list([str(i) for i in p.glob('*.jpg')])
        return [os.path.join(path, i) for i in os.listdir(path)]
    
    def get_output_size(self):
        return len(self.LABELS) + 1
    
    def char2idx(self, char):
        idx = self.LABELS.find(char.lower())
        return idx if idx != -1 else self.blank_label
    
    @staticmethod
    def labels_to_text(labels):
        ret = []
        for c in labels:
            if c == len(TextImageGenerator.LABELS):  # CTC Blank
                ret.append("")
            else:
                ret.append(TextImageGenerator.LABELS[c])
        return "".join(ret)
    
    def path2matrix(self, path):
        """
        input shape: (batch_size, w, h, channel)
        """
        img = cv2.imread(path)
        img = self.formatCaptcha(img)
        return img
    
    def formatCaptcha(self, img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = img/ 255.
#         img_transpose = np.einsum('hw->wh', img)
        img = np.expand_dims(img, axis=-1)
        return img
    
    def _get_one_captcha(self):
        captcha, text = gencaptcha.gen_one()
        return captcha, text

    def get_next_batch(self, paths, batch_size=32):
        def get_label(img_path):
            """
            获取验证码对应的字符串
            """
            return os.path.basename(img_path).split('.')[0].lower()
        i = 0
#         X_data = np.zeros((batch_size, self.img_w, self.img_h, self.channel))
        X_data = np.zeros((batch_size, self.img_h, self.img_w, self.channel))
        labels = np.zeros((batch_size, self.absolute_max_string_len))
        input_length = np.zeros([batch_size, 1])
        label_length = np.zeros([batch_size, 1])
        source_str = []
        while i < batch_size:
            if self.cur_idx >= len(paths):
                # 归零，洗牌
                self.cur_idx = 0
                np.random.shuffle(paths)
#             img_path = paths[self.cur_idx]
#             label_text = get_label(img_path)
            # 使用自动生成的
            captcha, label_text = self._get_one_captcha()
#             X_data[i, :] = self.path2matrix(img_path)
            X_data[i, :] = self.formatCaptcha(captcha)
            input_length[i] = self.img_w // self.downsample_factor - 2
            label_length[i] = len(label_text)
            labels[i] = [self.char2idx(char) for char in label_text]
            source_str.append(label_text)
            
            self.cur_idx += 1
            i += 1
            
        inputs = {
              'the_input': X_data,
              'the_labels': labels,
              'input_length': input_length,
              'label_length': label_length,
              'source_str': source_str  # used for visualization only
        }
        outputs = {'ctc': np.zeros([batch_size])}
        return (inputs, outputs)
            
    def get_next_train(self, batch_size=32):
        while True:
            yield self.get_next_batch(self.train_imgs, batch_size)
    
    def get_next_val(self, batch_size=100):
        while True:
            yield self.get_next_batch(self.validate_imgs, batch_size)
        


train_path = 'E:\\traindata\\captcha_create\\train'
validate_path = 'E:\\traindata\\captcha_create\\test'
test_img = os.path.join(train_path, '00ARLO.jpg')

img_w = 200
img_h = 60
channel = 1
downsample_factor = 4
img_gen = TextImageGenerator(train_path, validate_path, img_w, img_h, channel, downsample_factor)
ret_input, ret_output = next(img_gen.get_next_train(3))
ret_input, ret_output = next(img_gen.get_next_train(3))
print(ret_input['the_input'].shape)
print(ret_input['the_input'][:3, :3, :3])
print(ret_input['the_labels'])
print(ret_input['input_length'])
print(ret_input['label_length'])
print(ret_input['source_str'])

(3, 60, 200, 1)
[[[[0.89019608]
   [0.89019608]
   [0.89019608]]

  [[0.89019608]
   [0.89019608]
   [0.89019608]]

  [[0.89019608]
   [0.89019608]
   [0.89019608]]]


 [[[0.81960784]
   [0.81960784]
   [0.81960784]]

  [[0.81960784]
   [0.81960784]
   [0.81960784]]

  [[0.81960784]
   [0.81960784]
   [0.81960784]]]


 [[[0.77254902]
   [0.77254902]
   [0.77254902]]

  [[0.77254902]
   [0.77254902]
   [0.77254902]]

  [[0.77254902]
   [0.77254902]
   [0.77254902]]]]
[[12.  8.  7.  7. 32. 19.]
 [11. 19. 35. 21. 31.  8.]
 [ 3.  2. 11. 31.  3. 27.]]
[[48.]
 [48.]
 [48.]]
[[6.]
 [6.]
 [6.]]
['c877wj', 'bjzlv8', '32bv3r']


In [3]:
conv_filters = 16
kernel_size = (3, 3)
pool_size = 2
time_dense_size = 32
rnn_size = 512
minibatch_size = 32
OUTPUT_DIR = 'E:\\Workplace\\bdzh\\MachineLearning\\SmallCaptcha\\image_ocr'


# the actual loss calc occurs here despite it not being
# an internal Keras loss function

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

if K.image_data_format() == 'channels_first':
    input_shape = (channel, img_w, img_h)
else:
#     input_shape = (img_w, img_h, channel)
    input_shape = (img_h, img_w, channel)

act = 'relu'
input_data = Input(name='the_input', shape=input_shape, dtype='float32')
inner = Conv2D(conv_filters, kernel_size, padding='same',
               activation=act, kernel_initializer='he_normal',
               name='conv1')(input_data)
inner = MaxPooling2D(pool_size=(pool_size, pool_size), name='max1')(inner)
inner = Conv2D(conv_filters, kernel_size, padding='same',
               activation=act, kernel_initializer='he_normal',
               name='conv2')(inner)
inner = MaxPooling2D(pool_size=(pool_size, pool_size), name='max2')(inner)

conv_to_rnn_dims = (img_w // (pool_size ** 2), (img_h // (pool_size ** 2)) * conv_filters)
inner = Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)

# cuts down input size going into RNN:
inner = Dense(time_dense_size, activation=act, name='dense1')(inner)

# Two layers of bidirectional GRUs
# GRU seems to work as well, if not better than LSTM:
gru_1 = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)
gru_1b = GRU(rnn_size, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
gru1_merged = add([gru_1, gru_1b])
gru_2 = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
gru_2b = GRU(rnn_size, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b')(gru1_merged)

# transforms RNN output to character activations:
inner = Dense(img_gen.get_output_size(), kernel_initializer='he_normal',
              name='dense2')(concatenate([gru_2, gru_2b]))
y_pred = Activation('softmax', name='softmax')(inner)
base_model = Model(input=input_data, output=y_pred)
base_model.summary()

labels = Input(name='the_labels', shape=[img_gen.absolute_max_string_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
# Keras doesn't currently support loss funcs with extra parameters
# so CTC loss is implemented in a lambda layer
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

# clipnorm seems to speeds up convergence
sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)

model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)





Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 60, 200, 1)   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 60, 200, 16)  160         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 30, 100, 16)  0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 30, 100, 16)  2320        max1[0][0]                       
_____________________________________

E:\scriptpath\visualstudio\shared\Anaconda3_64\envs\tfgpu\lib\site-packages\ipykernel_launcher.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("th..., outputs=Tensor("so...)`


In [4]:

class VizCallback(keras.callbacks.Callback):

    def __init__(self, run_name, test_func, text_img_gen, num_display_words=6):
        self.test_func = test_func
        self.output_dir = os.path.join(
            OUTPUT_DIR, run_name)
        self.text_img_gen = text_img_gen
        self.num_display_words = num_display_words
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    def show_edit_distance(self, num):
        num_left = num
        mean_norm_ed = 0.0
        mean_ed = 0.0
        while num_left > 0:
            word_batch = next(self.text_img_gen)[0]
            num_proc = min(word_batch['the_input'].shape[0], num_left)
            decoded_res = decode_batch(self.test_func, word_batch['the_input'][0:num_proc])
            for j in range(num_proc):
                edit_dist = editdistance.eval(decoded_res[j], word_batch['source_str'][j])
                mean_ed += float(edit_dist)
                mean_norm_ed += float(edit_dist) / len(word_batch['source_str'][j])
            num_left -= num_proc
        mean_norm_ed = mean_norm_ed / num
        mean_ed = mean_ed / num
        print('\nOut of %d samples:  Mean edit distance: %.3f Mean normalized edit distance: %0.3f'
              % (num, mean_ed, mean_norm_ed))

    def on_epoch_end(self, epoch, logs={}):
        if epoch % 10 == 0: # 每10个周期计算一次正确率
            word_batch = next(self.text_img_gen)[0]
            res = decode_batch(self.test_func, word_batch['the_input'])
            if word_batch['the_input'][0].shape[0] < 256:
                cols = 2
            else:
                cols = 1
            acc = 0
            total = word_batch['the_input'].shape[0]
            for i in range(total):
                if word_batch['source_str'][i].lower() == res[i].lower():
                    acc += 1
            acc_ratio = 100 * acc / total
            print('正确率: %0.5f' % acc_ratio)
            if acc_ratio > 50:
                self.model.save_weights(os.path.join(self.output_dir, 'weights%02d_acc_%0.5f.h5' % (epoch, acc_ratio)))
        word_batch = next(self.text_img_gen)[0]
        res = decode_batch(self.test_func, word_batch['the_input'][0:self.num_display_words])
        if word_batch['the_input'][0].shape[0] < 256:
            cols = 2
        else:
            cols = 1
        for i in range(self.num_display_words):
            plt.subplot(self.num_display_words // cols, cols, i + 1)
            if K.image_data_format() == 'channels_first':
                the_input = word_batch['the_input'][i, 0, :, :]
            else:
                the_input = word_batch['the_input'][i, :, :, 0]
            plt.imshow(the_input, cmap='Greys_r')
            plt.xlabel('T = \'%s\' Decoed = \'%s\'' % (word_batch['source_str'][i], res[i]))
        plt.savefig(os.path.join(self.output_dir, 'e%02d.png' % (epoch)))
        plt.close()

def decode_batch(test_func, word_batch):
    out = test_func([word_batch])[0]
    ret = []
    for j in range(out.shape[0]):
        out_best = list(np.argmax(out[j, 2:], 1))
        out_best = [k for k, g in itertools.groupby(out_best)]
        outstr = TextImageGenerator.labels_to_text(out_best)
        ret.append(outstr)
    return ret

In [6]:



def evaluate(test_func, img_gen, batch_size=5):
    correct_count = 0
    img_gen = img_gen.get_next_val(batch_size=batch_size)
    _x, _ctc = next(img_gen)
    for i in range(batch_size):
        test_X = _x['the_input'][i]
        test_c = _x['source_str'][i]
        test_X = np.expand_dims(test_X, axis=-1)
        result = decode_batch(test_func, test_X)[0]
        try:
            if test_c.lower() == result.lower():
                correct_count += 1
                # print("[INFO] actual: %s, predict: %s" % (test_c, result))
            else:
                print("[ERROR] actual: %s, predict: %s" % (test_c, result))
        except Exception as e:
            print(e.message)
    print("Accuracy: %.2f%%" % ((float(correct_count) / batch_size) * 100))
    
class Evaluator(keras.callbacks.Callback):
    def __init__(self, test_func, text_img_gen):
        self.test_func = test_func
        self.text_img_gen = text_img_gen
        
    def on_epoch_end(self, epoch, logs={}):
        evaluate(self.test_func, self.text_img_gen)



In [8]:
# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)
# captures output of softmax so we can decode the output during visualization
test_func = K.function([input_data], [y_pred])

viz_cb = VizCallback(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'), test_func, img_gen.get_next_val())
# evaluate_gen = TextImageGenerator(train_path, validate_path, img_w, img_h, channel, downsample_factor)
# evaluator = Evaluator(test_func, evaluate_gen)

model.fit_generator(generator=img_gen.get_next_train(100),
                    steps_per_epoch=100,
                    epochs=400,
                    validation_data=img_gen.get_next_val(1000),
                    validation_steps=5,
                    callbacks=[viz_cb],
                    verbose=1)

Epoch 1/400
100/100 [==============================] - 88s 877ms/step - loss: 22.4953 - val_loss: 22.4608
正确率: 0.00000
Epoch 2/400
100/100 [==============================] - 65s 655ms/step - loss: 22.3769 - val_loss: 22.2498
Epoch 3/400
100/100 [==============================] - 63s 628ms/step - loss: 22.1305 - val_loss: 21.9076
Epoch 4/400
100/100 [==============================] - 63s 634ms/step - loss: 21.8595 - val_loss: 21.7784
Epoch 5/400
100/100 [==============================] - 63s 632ms/step - loss: 21.5581 - val_loss: 21.3423
Epoch 6/400
100/100 [==============================] - 63s 629ms/step - loss: 21.3541 - val_loss: 21.1599
Epoch 7/400
100/100 [==============================] - 63s 632ms/step - loss: 20.9416 - val_loss: 20.6329
Epoch 8/400
100/100 [==============================] - 63s 632ms/step - loss: 20.5281 - val_loss: 20.2903
Epoch 9/400
100/100 [==============================] - 64s 641ms/step - loss: 20.0753 - val_loss: 19.9676
Epoch 10/400
100/100 [===========

100/100 [==============================] - 63s 630ms/step - loss: 2.4318 - val_loss: 2.2447
Epoch 78/400
100/100 [==============================] - 63s 631ms/step - loss: 2.3249 - val_loss: 2.2628
Epoch 79/400
100/100 [==============================] - 63s 630ms/step - loss: 2.3344 - val_loss: 2.3537
Epoch 80/400
100/100 [==============================] - 63s 629ms/step - loss: 2.2352 - val_loss: 2.2917
Epoch 81/400
100/100 [==============================] - 63s 625ms/step - loss: 2.2004 - val_loss: 2.4512
正确率: 41.00000
Epoch 82/400
100/100 [==============================] - 62s 621ms/step - loss: 2.1517 - val_loss: 2.0160
Epoch 83/400
100/100 [==============================] - 63s 632ms/step - loss: 2.0798 - val_loss: 2.1729
Epoch 84/400
100/100 [==============================] - 64s 635ms/step - loss: 2.1045 - val_loss: 2.1390
Epoch 85/400
100/100 [==============================] - 64s 638ms/step - loss: 2.0342 - val_loss: 1.9650
Epoch 86/400
100/100 [==============================] 

100/100 [==============================] - 63s 631ms/step - loss: 1.0778 - val_loss: 1.2237
Epoch 154/400
100/100 [==============================] - 63s 632ms/step - loss: 1.0717 - val_loss: 1.1260
Epoch 155/400
100/100 [==============================] - 63s 629ms/step - loss: 1.0509 - val_loss: 1.0371
Epoch 156/400
100/100 [==============================] - 63s 630ms/step - loss: 1.0826 - val_loss: 1.1503
Epoch 157/400
100/100 [==============================] - 63s 629ms/step - loss: 1.0541 - val_loss: 0.9924
Epoch 158/400
100/100 [==============================] - 63s 629ms/step - loss: 1.0821 - val_loss: 0.9688
Epoch 159/400
100/100 [==============================] - 63s 626ms/step - loss: 1.1149 - val_loss: 1.2519
Epoch 160/400
100/100 [==============================] - 63s 630ms/step - loss: 1.0510 - val_loss: 1.0712
Epoch 161/400
100/100 [==============================] - 63s 630ms/step - loss: 1.0347 - val_loss: 0.9805
正确率: 77.00000
Epoch 162/400
100/100 [=======================

100/100 [==============================] - 62s 625ms/step - loss: 0.7835 - val_loss: 0.7649
Epoch 230/400
100/100 [==============================] - 63s 627ms/step - loss: 0.7626 - val_loss: 0.7504
Epoch 231/400
100/100 [==============================] - 62s 625ms/step - loss: 0.7094 - val_loss: 0.7550
正确率: 75.00000
Epoch 232/400
100/100 [==============================] - 62s 615ms/step - loss: 0.7232 - val_loss: 0.6876
Epoch 233/400
100/100 [==============================] - 62s 624ms/step - loss: 0.7250 - val_loss: 0.7167
Epoch 234/400
100/100 [==============================] - 62s 623ms/step - loss: 0.7449 - val_loss: 0.7329
Epoch 235/400
100/100 [==============================] - 63s 626ms/step - loss: 0.7292 - val_loss: 0.8047
Epoch 236/400
100/100 [==============================] - 62s 624ms/step - loss: 0.6931 - val_loss: 0.6603
Epoch 237/400
100/100 [==============================] - 62s 624ms/step - loss: 0.7252 - val_loss: 0.7143
Epoch 238/400
100/100 [=======================

100/100 [==============================] - 63s 625ms/step - loss: 0.5486 - val_loss: 0.5163
Epoch 306/400
100/100 [==============================] - 63s 626ms/step - loss: 0.5643 - val_loss: 0.5128
Epoch 307/400
100/100 [==============================] - 62s 625ms/step - loss: 0.5406 - val_loss: 0.4589
Epoch 308/400
100/100 [==============================] - 62s 624ms/step - loss: 0.5714 - val_loss: 0.5428
Epoch 309/400
100/100 [==============================] - 63s 626ms/step - loss: 0.5439 - val_loss: 0.5580
Epoch 310/400
100/100 [==============================] - 62s 622ms/step - loss: 0.5246 - val_loss: 0.5802
Epoch 311/400
100/100 [==============================] - 62s 624ms/step - loss: 0.5479 - val_loss: 0.5456
正确率: 79.00000
Epoch 312/400
100/100 [==============================] - 62s 615ms/step - loss: 0.5199 - val_loss: 0.5951
Epoch 313/400
100/100 [==============================] - 63s 625ms/step - loss: 0.5935 - val_loss: 0.4999
Epoch 314/400
100/100 [=======================

100/100 [==============================] - 63s 629ms/step - loss: 0.4873 - val_loss: 0.4389
正确率: 87.00000
Epoch 382/400
100/100 [==============================] - 62s 620ms/step - loss: 0.4750 - val_loss: 0.4856
Epoch 383/400
100/100 [==============================] - 63s 631ms/step - loss: 0.4595 - val_loss: 0.4734
Epoch 384/400
100/100 [==============================] - 63s 629ms/step - loss: 0.4890 - val_loss: 0.4139
Epoch 385/400
100/100 [==============================] - 63s 632ms/step - loss: 0.4215 - val_loss: 0.3577
Epoch 386/400
100/100 [==============================] - 63s 627ms/step - loss: 0.4357 - val_loss: 0.4341
Epoch 387/400
100/100 [==============================] - 63s 629ms/step - loss: 0.4804 - val_loss: 0.5241
Epoch 388/400
100/100 [==============================] - 63s 629ms/step - loss: 0.4719 - val_loss: 0.3929
Epoch 389/400
100/100 [==============================] - 63s 630ms/step - loss: 0.4958 - val_loss: 0.4135
Epoch 390/400
100/100 [=======================

In [ ]:
%matplotlib inline
generator = img_gen.get_next_val(2)
_x_input, _  = next(generator)
_X_test = _x_input['the_input']
_y_test = _x_input['the_labels']

img = _X_test[0]
print(img.shape)
print(np.einsum('hwc->whc', img).shape)
print(img[:, :, 0].T.shape)
print(np.max(img.flatten()))
print(np.min(img.flatten()))
print(_X_test[0].flatten().shape)
plt.subplot(1, 2, 1)
plt.imshow(img[:, :, 0].T)
plt.subplot(1, 2, 2)
img = np.einsum('hwc->whc', img)
plt.imshow(img)
plt.show()

# for i in range(_y_pred.shape[0]):
#     _y_pred = base_model.predict(np.expand_dims(_X_test[i], axis=0))[:, 2:, :]
#     print(_y_pred.shape)
#     for i in range(_y_pred.shape[0]):
#         __X_test = _X_test[i]
#         __y_test = _y_test[i]
#         __y_pred = _y_pred[i]
#         __y_pred = np.expand_dims(__y_pred, axis=0)
#         shape = __y_pred.shape
#         ctc_decode = K.ctc_decode(__y_pred, 
#                                   input_length=np.ones(shape[0])*shape[1])[0][0]
#         out = K.get_value(ctc_decode)[0]
#         print(out)


In [ ]:

from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)
from IPython.display import Image
Image(filename='model.png')

In [ ]:
model.save_weights('./weights/my_model_weights.h5')

In [ ]:
import os
train_path = 'E:\\traindata\\captcha_create\\train'
validate_path = 'E:\\traindata\\captcha_create\\test'
test_img = os.path.join(train_path, '00ARLO.jpg')

img_w = 200
img_h = 60
channel = 3
downsample_factor = 4
gen = TextImageGenerator(train_path, validate_path, img_w, img_h, channel, downsample_factor)
for i in range(40):
    next(gen.get_next_train(500))

imgs = next(gen.get_next_train(2))
_x, _ctc = imgs[0], imgs[1]

_the_input = _x['the_input']
_the_labels = _x['the_labels']
_input_length = _x['input_length']
_label_length = _x['label_length']
print(_ctc)
num = _the_input.shape[0]

for i in range(num):
    img = _the_input[i]
    print(img.shape)
    plt.subplot(1, num, i + 1)
    plt.xlabel('label: %s' % _the_labels[i])
    plt.imshow(img[:, :, 0], cmap='Greys_r')
    


In [9]:
model.save_weights('E:\\Workplace\\bdzh\\MachineLearning\\SmallCaptcha\\image_ocr\\2019_05_23_23_32_26\\weights400.h5')